In [95]:
from Bio import SeqIO
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from io import StringIO
from Bio.Blast import NCBIXML

In [96]:
input_folder = './015_Lazazzara_10-03-2022'
seq_folder = './sequences/'
output_folder = './consensus/' 
edit_folder = './edited/'
suffix = '.phd.1'

In [97]:
# getting the sequences from the sanger output (.phd.1 in this case)
fs = [f for f in os.listdir(input_folder) if f.endswith(suffix)] # collecting file names with sequencing information
qualdic = {} # collectiong information on filename : [base quality scores]
for f in fs:
    name = '_'.join(f.split('_')[0:2]) # file name
    sequence = '' # sequence variable
    qs = [] # quality scores list
    for row in open(input_folder+'/'+f).readlines()[103:-53]:
        row = row.strip('\n') # remove new lines \n
        row = row.split(' ') # make each row a list
        sequence += row[0] # the first list component is the name of the base; add it to the sequence variable
        qs.append(row[1]) # the second list component is the quality of the base; add it to the quality scores varaible
    # conditions to set the sequence description
    if '1' in name:
        des = 'Arboridia Puglia'
    elif '2' in name:
        des = 'Empoasca decipiens'
    elif '3' in name:
        des = 'Arboridia Creta'
    elif '4' in name:
        des = 'Arboridia Dalmazia'
    sequence = SeqRecord(Seq(sequence),id=name,description=des) # defining the SeqRecord object for the FASTA file write
    # write FASTA file
    with open(seq_folder+name+'.fa','w') as handle:
        SeqIO.write(sequence,handle,'fasta')
    qualdic[name] = qs # adding the filename : [base quality scores] to the disctionary

In [98]:
def get_sequence(f,s=0,e=0):
    f = SeqIO.read(f,'fasta')
    if e != 0:
        f.seq = f.seq[s:e]
    return f

In [130]:
# funciton input are:
# sequence1, sequence2, seq1_start, seq2_start, id seq1, id seq2, description, quality_dic, quality_check
def get_consensus(s1, s2, s1_s, s2_s, id1, id2, qs_d, n): 
    name = "".join([c for c in id1 if c in id2]) # get the name of the consensus
    q1 = qs_d[id1.split(' ')[0]] # get the quality of the sequence 1
    q2 = qs_d[id2.split(' ')[0]][::-1] # get the quality of the sequence 2
    s = ''
    for i in range(len(s1)):
        a = s1[i]
        b = s2[i]
        q_a = int(q1[s1_s+i])
        q_b = int(q2[s2_s+i])
        if a == b:
            s += a
        elif q_a > q_b and q_a > n:
            print(id1,a,q_a,id2,b,q_b, 'a')
            s += a
        elif q_b > q_a and q_b > n:
            print(id1,a,q_a,id2,b,q_b, 'b')
            s += b
        elif q_a > 20 and q_b > 20:
            print(id1,a,q_a,id2,b,q_b)
            if a == 'A':
                if b == 'G':
                    s += 'R'
                elif b == 'T':
                    s += 'W'
                elif b == 'C':
                    s += 'M'
                else:
                    s += 'N'
            if a == 'G':
                if b == 'A':
                    s += 'R'
                elif b == 'T':
                    s += 'K'
                elif b == 'C':
                    s += 'S'
                else:
                    s += 'N'
            if a == 'C':
                if b == 'G':
                    s += 'S'
                elif b == 'T':
                    s += 'Y'
                elif b == 'A':
                    s += 'M'
                else:
                    s += 'N'
            if a == 'T':
                if b == 'G':
                    s += 'K'
                elif b == 'A':
                    s += 'W'
                elif b == 'C':
                    s += 'Y'
                else:
                    s += 'N'
        else:
            print(id1,a,q_a,id2,b,q_b,'N')
            s += 'N'
    if '1' in name:
        des = 'Arboridia Puglia'
    elif '2' in name:
        des = 'Empoasca decipiens'
    elif '3' in name:
        des = 'Arboridia Creta'
    elif '4' in name:
        des = 'Arboridia Dalmazia'
    return SeqRecord(Seq(s),id=name,description=des)

In [131]:
# alignment of the two sequences (forward and reverse) using blast
fs = [f for f in os.listdir(seq_folder) if f.endswith('.fa')] # collecting file names with sequencing information
F = [x for x in fs if 'H' in x] # list of forward sequence files
R = [x for x in fs if 'L' in x] # list of reverse sequence files
l = list(zip(F,R)) # list of tuples (forward,reverse) of each sequence
for tup in l:
    s1,s2 = tup
    output = NcbiblastnCommandline(query=seq_folder+s1, subject=seq_folder+s2, outfmt=5)()[0]
    blast_result_record = NCBIXML.read(StringIO(output))
    q_start = 9999
    q_end = 0
    for alignment in blast_result_record.alignments:
        for hsp in alignment.hsps:
            query = hsp.query
            sbjct = hsp.sbjct
            q_start = min(hsp.query_start,q_start)
            q_end = max(hsp.query_end,q_end)
            if '-' in hsp.query:
                print(s1)
    seq1 = get_sequence(seq_folder+s1,q_start-1,q_end)
    with open(edit_folder+s1,'w') as handle:
        SeqIO.write(seq1,handle,'fasta')
    s_start = 9999
    s_end = 0
    for alignment in blast_result_record.alignments:
        for hsp in alignment.hsps:
            s_start = min(hsp.sbjct_end,s_start)
            s_end = max(hsp.sbjct_start,s_end)
        if '-' in hsp.sbjct:
                print(s2)
    seq2 = get_sequence(seq_folder+s2,s_start-1,s_end)
    with open(edit_folder+s2,'w') as handle:
        SeqIO.write(seq2,handle,'fasta')
    cons = get_consensus(query, sbjct, q_start-1, s_start-1, seq1.id, seq2.id, qualdic, 35)
    with open(output_folder+cons.id+'.fa','w') as handle:
        SeqIO.write(cons,handle,'fasta')

1A_H C 13 1A_L T 62 b
1A_H A 11 1A_L G 62 b
1D_H G 10 1D_L A 62 b
4A_H A 54 4A_L C 62 b
4C_H T 13 4C_L C 54 b
4C_H A 13 4C_L G 62 b
4D_H A 62 4D_L C 46 a
4E_H A 62 4E_L C 59 a


In [125]:
output = NcbiblastnCommandline(query='D:/PhD/PhD.AES.Dsuz.Riccardo.Piccinno/Cicaline/my_code/sequences/1A_H.fa', 
                               subject='D:/PhD/PhD.AES.Dsuz.Riccardo.Piccinno/Cicaline/my_code/sequences/1A_L.fa', 
                               outfmt=5)()[0]
blast_result_record = NCBIXML.read(StringIO(output))
q_start = 9999
q_end = 0
for alignment in blast_result_record.alignments:
    for hsp in alignment.hsps:
        print(hsp.query)
        print(hsp.match)
        print(hsp.sbjct)
        seq = hsp.query
        ids = [i for i in range(len(hsp.match)) if hsp.match[i] != '|']
        for el in ids:
            a = hsp.query[el]
            b = hsp.sbjct[el]
            if a == 'A':
                if b == 'G':
                    seq = seq[:el] + 'R' + seq[el+1:]
                elif b == 'T':
                    seq = seq[:el] + 'W' + seq[el+1:]
                elif b == 'C':
                    seq = seq[:el] + 'M' + seq[el+1:]
                else:
                    seq = seq[:el] + 'N' + seq[el+1:]
            if a == 'G':
                if b == 'A':
                    seq = seq[:el] + 'R' + seq[el+1:]
                elif b == 'T':
                    seq = seq[:el] + 'K' + seq[el+1:]
                elif b == 'C':
                    seq = seq[:el] + 'S' + seq[el+1:]
                else:
                    seq = seq[:el] + 'N' + seq[el+1:]
            if a == 'C':
                if b == 'G':
                    seq = seq[:el] + 'S' + seq[el+1:]
                elif b == 'T':
                    seq = seq[:el] + 'Y' + seq[el+1:]
                elif b == 'A':
                    seq = seq[:el] + 'M' + seq[el+1:]
                else:
                    seq = seq[:el] + 'N' + seq[el+1:]
            if a == 'T':
                if b == 'G':
                    seq = seq[:el] + 'K' + seq[el+1:]
                elif b == 'A':
                    seq = seq[:el] + 'W' + seq[el+1:]
                elif b == 'C':
                    seq = seq[:el] + 'Y' + seq[el+1:]
                else:
                    seq = seq[:el] + 'N' + seq[el+1:]
            print(hsp.query_start + el, qualdic['1A_H'][hsp.query_start + el-1], qualdic['1A_L'][-(hsp.query_start +el)])

ATAGCACCGGCCAAAACTGGCAAAGATAATAAAAGTAAAATTGCTGTAATTACTACTGACCATACAAATAATGGAATTCGATCTAAACTCAAACTAATTGAACGCATGTTGATAACTGTAGTAATAAAATTTACTGCTCCCAGGATAGATGAAATCCCAGCTAAATGTAATGAAAAGATTGCTAAATCTACTCTTGCCCCAGAATGGGCAATATTAGATGAAAGAGGGGGGTAAACTGTTCATCCGGTTCCAGCCCCTGTTTCAATCATAGATCTCGTTAGTAATAAAGTTAATGAAGGAGGTAAAAGTCAAAATCTTATATTATTTATTCGGGGAAAAGCTATATCAGGGGCCCCAATCATTAAAGGTAAAAGTCAATTACCAAAACCCCCGATCATAATCGGCATTACTATAAAGAAAATTATAATAAATGCATGAGATGTTACAATAACATTATAAATTTGATCATTGTTTAAAAAGACC
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
ATAGCACCGGCCAAAACTGGCAAAGATAATAA